# How to compute CO2 emission

To know the amount of estimated CO2 emissions associated to the execution of our applications is a crucial step to realize our environmental impact. To this end, we show here how to accomplish this using the [CodeCarbon](https://mlco2.github.io/codecarbon/index.html) library/service.

This notebook has been developed by the [SINAI](https://sinai.ujaen.es) research group for its usage in the MentalRiskES evaluation campaign at IberLEF 2023 and 2024.



## Install CodeCarbon package

In [3]:
!pip install codecarbon

## Estimate different impact metrics

There are several ways to track code's emissions. Here we will use start() and stop() approach.

## Online mode
If the server where you make the predictions has an internet connection, the library automatically obtains the iso code of the country where the server is located.

In [13]:
from codecarbon import EmissionsTracker

config = {
    "save_to_file": True,
    "log_level": "DEBUG",
    "tracking_mode": "process",
    "output_dir": ".",
}

tracker = EmissionsTracker(**config)

[codecarbon INFO @ 11:57:28] [setup] RAM Tracking...
[codecarbon INFO @ 11:57:28] [setup] GPU Tracking...
[codecarbon INFO @ 11:57:28] No GPU found.
[codecarbon INFO @ 11:57:28] [setup] CPU Tracking...
[codecarbon DEBUG @ 11:57:28] Not using PowerGadget, an exception occurred while instantiating IntelPowerGadget : Platform not supported by Intel Power Gadget
[codecarbon DEBUG @ 11:57:28] Not using the RAPL interface, an exception occurred while instantiating IntelRAPL : Intel RAPL files not found at /sys/class/powercap/intel-rapl on linux
[codecarbon DEBUG @ 11:57:28] Not using PowerMetrics, an exception occurred while instantiating Powermetrics : Platform not supported by Powermetrics
[codecarbon WARNING @ 11:57:28] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 11:57:30] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 11:57:30] CPU Model on constant consumption mode: Intel(R) Xeon(

## Offline mode
If the server where the predictions are made does not have an internet connection, please use the offline mode and indicate the country where the server is located.

In [ ]:
from codecarbon import OfflineEmissionsTracker

config = {
    "save_to_file": True,
    "log_level": "DEBUG",
    "tracking_mode": "process",
    "output_dir": ".",
    "country_iso_code":"ESP"
}

tracker = OfflineEmissionsTracker(**config)

Now, we will compute the CO2 cost of translate some text into Spanish using deep learning models.

In [6]:
!pip install transformers sentencepiece

In [14]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline, set_seed
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-es")
translator = pipeline('translation', tokenizer=tokenizer, model=model)
set_seed(42)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [16]:
tracker.start()
translation = translator("After the breakfast, we will go for lunch...", num_return_sequences=2)
emissions = tracker.stop()

[codecarbon WARNING @ 11:57:43] Already started tracking
[codecarbon WARNING @ 11:57:44] Tracker already stopped !
[codecarbon INFO @ 11:57:44] Energy consumed for RAM : 0.000001 kWh. RAM Power : 0.6752715110778809 W
[codecarbon DEBUG @ 11:57:44] RAM : 0.68 W during 3.57 s [measurement time: 0.0048]
[codecarbon INFO @ 11:57:44] Energy consumed for all CPUs : 0.000049 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 11:57:44] CPU : 42.50 W during 3.58 s [measurement time: 0.0023]
[codecarbon INFO @ 11:57:44] 0.000050 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 11:57:44] last_duration=3.574281930923462
------------------------
[codecarbon DEBUG @ 11:57:44] EmissionsData(timestamp='2024-02-19T11:57:44', project_name='codecarbon', run_id='21644205-e330-4f6a-a853-cc833f578b64', duration=4.207031488418579, emissions=1.7560302235649154e-05, emissions_rate=4.174036320857218e-06, cpu_power=42.5, gpu_power=0.0, ram_power=0.6752715110778809, cpu_energy=4.949967083003786e-05,

In [17]:
print(translation)
print(emissions)

[{'translation_text': 'Después del desayuno, iremos a almorzar...'}, {'translation_text': 'Después del desayuno, iremos a comer...'}]
1.7560302235649154e-05


# Data to be submitted

Collected measurements are detailed [here](https://mlco2.github.io/codecarbon/output.html)

MentalRiskES requires of the following info to be added to every submission under a "impact" entry in the JSON object.

In [18]:
relevant_cols = [
    "duration", "emissions", "cpu_energy", "gpu_energy", "ram_energy",
    "energy_consumed", "cpu_count", "gpu_count", "cpu_model", "gpu_model",
    "ram_total_size","country_iso_code"
]

In order to get all the information from the last entry in the logged CSV file, we can use a Pandas dataframe and convert last row to a dictionary.


In [19]:
import pandas as pd

df = pd.read_csv("emissions.csv")
measurements = df.iloc[-1][relevant_cols].to_dict()

measurements # This is what your team must send in the POST request

{'duration': 4.207031488418579,
 'emissions': 1.7560302235649154e-05,
 'cpu_energy': 4.949967083003786e-05,
 'gpu_energy': 0,
 'ram_energy': 7.837895592192959e-07,
 'energy_consumed': 5.028346038925715e-05,
 'cpu_count': 2,
 'gpu_count': nan,
 'cpu_model': 'Intel(R) Xeon(R) CPU @ 2.20GHz',
 'gpu_model': nan,
 'ram_total_size': 12.674781799316406,
 'country_iso_code': 'USA'}